# Mondrian Multidimensional K-Anonimity
**Authors:** Giacomo Garbarino S4545532 - Manuel Parmiggiani S4701853

In [1]:
import pandas as pd
import numpy as np
import copy

Global variables :

In [2]:
# dictionary column name -> column type
column_type_dict = {}
# dictionary element -> hierarchy of that element
hierarchy_dict = {}
# value of K
K = 2

Given a DataFrame, returns the subset of its columns that are Quasi Identifiers columns :

In [3]:
def select_qi_columns(df):
    
    columns = df.columns.tolist()
    
    for column in columns:
        if column not in column_type_dict:
            columns.remove(column)
            continue
        if column_type_dict[column] not in ["NUMERICAL","CATEGORICAL"]:
            columns.remove(column)
    
    return columns

Given a DataFrame (group) the column with the highest cardinality :

In [4]:
def get_split_column(group, columns):
    
    cardinality_dict = {column : len(np.unique(group[column])) for column in columns}
    split_column = max(cardinality_dict, key = cardinality_dict.get)
    
    return split_column

Given a list of values, returns the nearest to their mean :

In [5]:
def get_numerical_threshold(values):
    
    mean = round(sum(values)/len(values))
    threshold = min(values, key = lambda x : abs(x-mean))
    
    return threshold

Given a list of values, returns an interval that contains all of them :

In [6]:
def create_interval(values):
    
    interval_string = "[" + str(min(values)) + "-" + str(max(values)) + "]"
    
    return interval_string

Given a list of labels (intended as labels of nodes of the hierarchy tree), returns their first common ancestor : 

In [7]:
def get_upper_bound(labels):
    
    first_label = labels[0]
    
    # for each value in the hierarchy of the first label...
    for elem in hierarchy_dict[first_label]:
        is_elem_in_common = True
        for another_label in labels[1:]:
            # ...chek if all of the other labels have that value in their hierarchies
            if elem not in hierarchy_dict[another_label]:
                is_elem_in_common = False
                break
        # the first one in common is the correct one, so it's returned
        # (always found if the hierarchy.txt file is well defined)
        if is_elem_in_common:
            return elem

Given an element of a hierarchy, it returns all of its direct children nodes:

In [8]:
def get_children(element):
    
    children_list = []
    
    for key in hierarchy_dict:
        hierarchy = hierarchy_dict[key]
        if element not in hierarchy:
            continue
        # for each hierarchy containing the element...
        for e in hierarchy:
            # ...find the position of the element...
            if e == element :
                # ...and add the previous one (that is a child node of the current one) to the children list
                child = hierarchy[hierarchy.index(element)-1]
                if hierarchy.index(element) > hierarchy.index(child):
                    children_list.append(child)
                break

    return np.unique(children_list).tolist()

Given an element, returns its ancestor (chosen among a list of possible ones) :

In [9]:
def get_ancestor_of(element, possible_ancestors_list):
    
    # only one ancestor in the list is the right one, so the first one found is returned
    for elem in hierarchy_dict[element]:
        if elem in possible_ancestors_list:
            return elem

Splits a group of rows in 2 or more sub-groups, according to the values they have for a given split column :

In [10]:
def split_by_column(group, split_column):
    
    global column_type_dict
    
    if column_type_dict[split_column] == "NUMERICAL":
        # a threshold is calulated in order to split the group in two
        threshold = get_numerical_threshold(group[split_column].tolist())
        group1 = group[group[split_column] > threshold]
        group2 = group[group[split_column] < threshold]
        # the leftover is divided among the two groups
        leftover = group[group[split_column] == threshold]
        for _,row in leftover.iterrows():
            if (len(group1) < len(group2)):
                group1 = group1.append(row)
            else :
                group2 = group2.append(row)
        return [group1, group2]
    
    else:
        # the upper bound for the column values is calculated...
        threshold = get_upper_bound(group[split_column].tolist())
        # ...and the children nodes of the upper bound are extracted
        children_list = get_children(threshold)
        # if the upper bound is a leaf, then the group itself it's already minimal for this column
        if len(children_list) == 0:
            return [group]
        
        new_groups_dict = {}
        leftover = []
        i=0
        # assigning each column to the group it will belong (one group for each children node)
        for _,row in group.iterrows():
            key = get_ancestor_of(row[split_column], children_list)
            if key == None:
                leftover.append(i)
                i = i+1
                continue
            if key not in new_groups_dict:
                new_groups_dict[key] = []
            new_groups_dict[key].append(i)
            i = i+1
        # elements in 'leftover' are the ones with the column value equal to the upper bound
        # since they can fit in each of the new groups, they are distribuited among them...
        for index in leftover:
            length_dict = {key : len(new_groups_dict[key]) for key in new_groups_dict}
            # ...following the rule of being assigned to the smallest group each
            lowest = min(length_dict, key = length_dict.get)
            new_groups_dict[lowest].append(index)
            
        new_groups_list = []
        for child in children_list:
            # the new group is created extracting the rows from the original group...
            if child in new_groups_dict:
                new_group = group.iloc[new_groups_dict[child]]
                if not new_group.empty:
                    # ...then is added to the list cotaining all of the groups
                    new_groups_list.append(new_group)
                
        return new_groups_list

Splits a given DataFrame into groups of rows :

In [11]:
def split(df):
    
    global K
    columns = select_qi_columns(df)
    group_list = []
    
    while columns:
        split_column = get_split_column(df, columns)
        new_groups = split_by_column(df, split_column)
        
        # checks over the list of new groups :
        #---the list must be not empty
        #---all of the new groups must contain at least K rows
        #---none of the new groups can be equal to the original one
        
        # if all of these requirements are satisfied...
        if len(new_groups) > 0 and np.all([len(g.index) >= K for g in new_groups]) and np.all([not df.equals(g) for g in new_groups]):
            # ...the new groups are added to the tail of the group list
            for new_group in new_groups:
                new_split = split(new_group)
                if type(new_split) == list:
                    group_list.extend(new_split)
                else:
                    group_list.append(new_split)
            return group_list
        # if not...
        else:
            # ...no more splits are possible for the chosen split_column
            columns.remove(split_column)
            continue
        break
    
    return df

Anonymizes a list of groups and returns the final anonymized DataFrame :

In [12]:
def anonymize(group_list):
    
    final_group_list = copy.deepcopy(group_list)
    
    # for each group...
    for group in final_group_list:
        
        # ...and for each column...
        for column in group.columns:
            
            # ...the value of each row of the group for that column is replaced with a
            #    new value representing all of that column values of the currrent group
            
            if column_type_dict[column] == "NUMERICAL":
                group.loc[:, column] = create_interval(group[column].tolist())
                
            elif column_type_dict[column] == "CATEGORICAL":
                new_value = get_upper_bound(group[column].tolist())
                if len(np.unique(group[column])) == 1 and len(hierarchy_dict[new_value]) > 1:
                    new_value = hierarchy_dict[new_value][1]
                group.loc[:, column] = new_value
    
    final_df = pd.concat(final_group_list)
    return final_df

# Start

In [13]:
column_file = open("Data/column_types.txt")
for line in column_file:
    values_list = line.split(",")
    #the last element of each row may contain a special \n character we want to remove
    first = values_list[0]
    last = values_list.pop()
    if (last[len(last) - 1] == "\n"):
        last = last[:len(last) - 1]
    #then the formatted value is re-added to the list
    column_type_dict[first] = last

print(column_type_dict)

{'Age': 'NUMERICAL', 'Birthplace': 'CATEGORICAL', 'Occupation': 'CATEGORICAL', 'Annual Income': 'SD'}


In [14]:
hierarchy_file = open("Data/hierarchies.txt")
for line in hierarchy_file:
    if (line[0] == '-'):
        continue
    values_list = line.split(",")
    #the last element of each row may contain a special \n character we want to remove
    last = values_list.pop()
    if (last[len(last) - 1] == "\n"):
        last = last[:len(last) - 1]
    #then the formatted value is re-added to the list
    values_list.append(last)
    for index,value in enumerate(values_list):
        hierarchy_dict[value] = values_list[index:]
    
print (hierarchy_dict)

{'Cuba': ['Cuba', 'North America', 'America', 'World'], 'North America': ['North America', 'America', 'World'], 'America': ['America', 'World'], 'World': ['World'], 'Jamaica': ['Jamaica', 'North America', 'America', 'World'], 'Canada': ['Canada', 'North America', 'America', 'World'], 'Columbia': ['Columbia', 'South America', 'America', 'World'], 'South America': ['South America', 'America', 'World'], 'Ecuador': ['Ecuador', 'South America', 'America', 'World'], 'England': ['England', 'Central Europe', 'Europe', 'World'], 'Central Europe': ['Central Europe', 'Europe', 'World'], 'Europe': ['Europe', 'World'], 'Portugal': ['Portugal', 'West Europe', 'Europe', 'World'], 'West Europe': ['West Europe', 'Europe', 'World'], 'Ireland': ['Ireland', 'West Europe', 'Europe', 'World'], 'Japan': ['Japan', 'East Asia', 'Asia', 'World'], 'East Asia': ['East Asia', 'Asia', 'World'], 'Asia': ['Asia', 'World'], 'Teacher': ['Teacher', 'Scholastic', 'Any'], 'Scholastic': ['Scholastic', 'Any'], 'Any': ['Any'

In [15]:
dataset = pd.read_csv("Data/data.csv")
print(dataset)
split_dataset = split(dataset)
for d in split_dataset:
    print("\nSPLIT")
    print(d)

    Age     Birthplace          Occupation  Annual Income
0    21        Jamaica          Scholastic         100000
1    28       Columbia               State          30000
2    24           Cuba  University Teacher          40000
3    23        Ecuador         Post Office          50000
4    28         Canada              Police          20000
5    45       Portugal             Private          30000
6    43        Ireland              Office          34000
7    45          Japan              Police          60000
8    27          Japan         Post Office          25000
9    33  North America         Post Office          30000
10   32    West Europe              Office          40000

SPLIT
   Age     Birthplace   Occupation  Annual Income
4   28         Canada       Police          20000
9   33  North America  Post Office          30000

SPLIT
   Age Birthplace          Occupation  Annual Income
2   24       Cuba  University Teacher          40000
0   21    Jamaica          Scholas

In [16]:
final_dataset = anonymize(split_dataset)
print(final_dataset)

        Age     Birthplace  Occupation  Annual Income
4   [28-33]  North America       State          20000
9   [28-33]  North America       State          30000
2   [21-24]  North America  Scholastic          40000
0   [21-24]  North America  Scholastic         100000
3   [23-28]  South America       State          50000
1   [23-28]  South America       State          30000
7   [27-45]      East Asia       State          60000
8   [27-45]      East Asia       State          25000
6   [32-45]    West Europe     Private          34000
10  [32-45]    West Europe     Private          40000
5   [32-45]    West Europe     Private          30000


In [17]:
final_dataset.sort_index().to_csv("Data/anonymized_data.csv", index=False)